# Load Dataset

In [1]:
!pip install -q datasets
!pip install dahuffman
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import ViTModel
import numpy as np
import torch
import cv2
import torch.nn as nn
from transformers import ViTModel, ViTConfig
from torchvision import transforms
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import torch.nn as nn
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import numpy as np
from collections import Counter
import torch
from collections import Counter
from dahuffman import HuffmanCodec



In [3]:
dataset = load_dataset("Matthijs/snacks")
print(dataset)
print(dataset["train"].features['label'].names)

labels = dataset["train"].features["label"].names
num_labels = len(dataset["train"].features["label"].names)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label
print(label2id)
print(id2label)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 4838
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 952
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 955
    })
})
['apple', 'banana', 'cake', 'candy', 'carrot', 'cookie', 'doughnut', 'grape', 'hot dog', 'ice cream', 'juice', 'muffin', 'orange', 'pineapple', 'popcorn', 'pretzel', 'salad', 'strawberry', 'waffle', 'watermelon']
{'apple': 0, 'banana': 1, 'cake': 2, 'candy': 3, 'carrot': 4, 'cookie': 5, 'doughnut': 6, 'grape': 7, 'hot dog': 8, 'ice cream': 9, 'juice': 10, 'muffin': 11, 'orange': 12, 'pineapple': 13, 'popcorn': 14, 'pretzel': 15, 'salad': 16, 'strawberry': 17, 'waffle': 18, 'watermelon': 19}
{0: 'apple', 1: 'banana', 2: 'cake', 3: 'candy', 4: 'carrot', 5: 'cookie', 6: 'doughnut', 7: 'grape', 8: 'hot dog', 9: 'ice cream', 10: 'juice', 11: 'muffin', 12: 'orange', 13: 'pineapple', 14: 'popcorn', 15: 'pretzel'

# Forward Pass ViT Model with a Toy Image

In [4]:
# Create toy image with dim (batch x channel x width x height)
toy_img = torch.rand(1, 3, 48, 48)

# Define conv layer parameters
num_channels = 3
hidden_size = 768
patch_size = 16

projection = nn.Conv2d(num_channels, hidden_size, kernel_size=patch_size,
             stride=patch_size)

out_projection = projection(toy_img)

print(f'Original image size: {toy_img.size()}')
print(f'Size after projection: {out_projection.size()}')


patch_embeddings = out_projection.flatten(2).transpose(1, 2)
print(f'Patch embedding size: {patch_embeddings.size()}')

# Define [CLS] token embedding with the same emb dimension as the patches
batch_size = 1
cls_token = nn.Parameter(torch.randn(1, 1, hidden_size))
cls_tokens = cls_token.expand(batch_size, -1, -1)

# Prepend [CLS] token in the beginning of patch embedding
patch_embeddings = torch.cat((cls_tokens, patch_embeddings), dim=1)
print(f'Patch embedding size: {patch_embeddings.size()}')

# Define position embedding with the same dimension as the patch embedding
position_embeddings = nn.Parameter(torch.randn(batch_size, 10, hidden_size))

# Add position embedding into patch embedding
input_embeddings = patch_embeddings + position_embeddings
print(f'Input embedding size: {input_embeddings.size()}')


# Define parameters for ViT-base (example)
num_heads = 12
num_layers = 12

# Define Transformer encoders' stack
transformer_encoder_layer = nn.TransformerEncoderLayer(
           d_model=hidden_size, nhead=num_heads,
           dim_feedforward=int(hidden_size * 4),
           dropout=0.1)
transformer_encoder = nn.TransformerEncoder(
           encoder_layer=transformer_encoder_layer,
           num_layers=num_layers)

# Forward pass
output_embeddings = transformer_encoder(input_embeddings)
print(f' Output embedding size: {output_embeddings.size()}')

# Load pretrained model
model_checkpoint = 'google/vit-base-patch16-224-in21k'
model = ViTModel.from_pretrained(model_checkpoint, add_pooling_layer=False)

# Example input image
input_img = torch.rand(batch_size, num_channels, 224, 224)

# Forward pass input image
output_embedding = model(input_img)
print(output_embedding)
print(f"Ouput embedding size: {output_embedding['last_hidden_state'].size()}")

num_labels = 20

# Define linear classifier layer
classifier = nn.Linear(hidden_size, num_labels)

# Forward pass on the output embedding of [CLS] token
output_classification = classifier(output_embedding['last_hidden_state'][:, 0, :])
print(f"Output embedding size: {output_classification.size()}")

Original image size: torch.Size([1, 3, 48, 48])
Size after projection: torch.Size([1, 768, 3, 3])
Patch embedding size: torch.Size([1, 9, 768])
Patch embedding size: torch.Size([1, 10, 768])
Input embedding size: torch.Size([1, 10, 768])
 Output embedding size: torch.Size([1, 10, 768])


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.1093, -0.1853,  0.0527,  ...,  0.1627,  0.0325, -0.3765],
         [ 0.2186,  0.0706,  0.0383,  ...,  0.0539,  0.1203, -0.2467],
         [ 0.0867, -0.1921,  0.0217,  ..., -0.0766,  0.1316, -0.4392],
         ...,
         [ 0.1484, -0.0600, -0.1176,  ..., -0.0161,  0.1460, -0.3408],
         [ 0.0706, -0.1715,  0.0551,  ...,  0.0265,  0.1014, -0.4343],
         [ 0.1647, -0.0807,  0.0622,  ..., -0.0121,  0.0207, -0.2765]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=None, hidden_states=None, attentions=None)
Ouput embedding size: torch.Size([1, 197, 768])
Output embedding size: torch.Size([1, 20])


# Fine Tuning ViT Impelementation

In [5]:
#Pretrained model checkpoint
model_checkpoint = 'google/vit-base-patch16-224-in21k'

In [6]:
class ImageDataset(torch.utils.data.Dataset):

  def __init__(self, input_data):

      self.input_data = input_data
      self.transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224, 224), antialias=True),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5])
        ])

  def __len__(self):
      return len(self.input_data)

  def get_images(self, idx):
      return self.transform(self.input_data[idx]['image'])

  def get_labels(self, idx):
      return self.input_data[idx]['label']

  def __getitem__(self, idx):
      # Get input data in a batch
      train_images = self.get_images(idx)
      train_labels = self.get_labels(idx)

      return train_images, train_labels

In [7]:
def train_model(model, train_dataset, epochs, learning_rate, batch_size):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    model = model.to(device)

    train_loader = DataLoader(ImageDataset(train_dataset), batch_size=batch_size, shuffle=True, num_workers=1)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        model.train()
        total_loss, total_acc = 0, 0

        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

        avg_loss = total_loss / len(train_loader.dataset)
        avg_acc = total_acc / len(train_loader.dataset)
        print(f'Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return model

def test_model(model, test_dataset, batch_size):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    model = model.to(device)

    test_loader = DataLoader(ImageDataset(test_dataset), batch_size=batch_size, shuffle=False, num_workers=1)

    model.eval()
    criterion = nn.CrossEntropyLoss().to(device)

    total_loss, total_acc = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

    avg_loss = total_loss / len(test_loader.dataset)
    avg_acc = total_acc / len(test_loader.dataset)
    print(f'Test - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return avg_loss, avg_acc

In [8]:
import torch
import torch.nn as nn
from transformers import ViTModel, ViTConfig

class ViT(nn.Module):
    def __init__(self, config=ViTConfig(), num_labels=20,
                 model_checkpoint='google/vit-base-patch16-224-in21k',
                 encoded_weights=None, codec=None):
        super(ViT, self).__init__()
        self.vit = ViTModel.from_pretrained(model_checkpoint, add_pooling_layer=False)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.encoded_weights = encoded_weights
        self.codec = codec

    def forward(self, x):
        x = self.vit(x)['last_hidden_state']
        output = self.classifier(x[:, 0, :])
        return output


In [9]:
# Hyperparameters
EPOCHS = 3
LEARNING_RATE = 1e-4
BATCH_SIZE = 32
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")



In [ ]:
model = ViT().to(device)
trained_model = train_model(model, dataset['train'], EPOCHS, LEARNING_RATE, BATCH_SIZE)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 152/152 [02:40<00:00,  1.06s/it]


Epoch 1/3 - Loss: 0.0446, Accuracy: 0.8295


 68%|██████▊   | 104/152 [01:59<00:55,  1.15s/it]

In [11]:
# load weights if you have it locally so don't have to fine tune again

trained_model =  ViT()
#trained_model.load_state_dict(torch.load('original_model.pth'))

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
test_loss, test_accuracy = test_model(trained_model, dataset['test'], BATCH_SIZE)
print(test_loss, test_accuracy)

100%|██████████| 30/30 [00:12<00:00,  2.42it/s]

Test - Loss: 0.0943, Accuracy: 0.0557
0.09428719612730652 0.05567226890756303


In [13]:
ViT()

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ViT(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072,

# Round model weights

In [14]:
import copy


trained_model.eval()  # Set the model to evaluation mode
model.to('cpu')  # Move the model to CPU

rounded_model = copy.deepcopy(trained_model)
rounded_model.to('cpu')
precision = 2

def round_with_decimal_precision(tensor, decimal_precision=2):
    # Convert the PyTorch tensor to a NumPy array
    numpy_array = tensor.numpy()

    # Apply rounding with decimal precision using NumPy
    rounded_numpy_array = np.round(numpy_array, decimals=decimal_precision)

    # Convert the rounded NumPy array back to a PyTorch tensor
    rounded_tensor = torch.tensor(rounded_numpy_array, dtype=tensor.dtype)

    return rounded_tensor


for name, param in rounded_model.named_parameters():
    param.data = round_with_decimal_precision(param.data, decimal_precision=precision).to(torch.float16)
    print(param)
    print(param.dtype)


Streaming output truncated to the last 5000 lines.
        0.4399, 0.3999, 0.3401, 0.4099, 0.4399, 0.4099, 0.4099, 0.3899, 0.4199,
        0.3201, 0.3701, 0.4199, 0.3799, 0.3799, 0.3899, 0.3799, 0.3999, 0.3999,
        0.3999, 0.3999, 0.3799, 0.4199, 0.4199, 0.4199, 0.3601, 0.4099, 0.3999,
        0.4099, 0.3799, 0.4099, 0.4500, 0.3999, 0.3899, 0.3999, 0.3899, 0.3999,
        0.4500, 0.4099, 0.3999, 0.4099, 0.4099, 0.4099, 0.4099, 0.4299, 0.4099,
        0.4199, 0.4199, 0.4199, 0.3999, 0.3899, 0.4199, 0.3601, 0.3999, 0.4199,
        0.3999, 0.3999, 0.3201, 0.3899, 0.4199, 0.4399, 0.3999, 0.4099, 0.4199,
        0.3999, 0.4099, 0.3999, 0.3899, 0.3999, 0.4099, 0.3799, 0.3601, 0.4399,
        0.4099, 0.3899, 0.4299, 0.1000, 0.4199, 0.3799, 0.4099, 0.3899, 0.4099,
        0.4099, 0.3799, 0.3201], dtype=torch.float16, requires_grad=True)
torch.float16
Parameter containing:
tensor([ 0.1200,  0.1801, -0.1000,  0.0300, -0.0100, -0.0100,  0.0200, -0.1000,
         0.0200, -0.1400, -0.0100,  0.0

In [15]:
print("Original Weights:")
for param in trained_model.parameters():
    print(param.data)
    print(param.dtype)

Streaming output truncated to the last 5000 lines.
         4.7759e-02,  4.0755e-02, -2.9920e-02,  2.0163e-02,  2.9407e-02,
        -3.7971e-02,  2.4920e-02, -1.8576e-02], device='cuda:0')
torch.float32
tensor([[ 0.1928, -0.0868, -0.0446,  ..., -0.0517,  0.0030, -0.0243],
        [-0.0062, -0.1702, -0.0560,  ...,  0.0526,  0.1077,  0.0148],
        [ 0.0665,  0.0654, -0.0058,  ...,  0.0854,  0.0346, -0.0234],
        ...,
        [-0.1057,  0.1022,  0.1011,  ...,  0.0755,  0.0499, -0.0159],
        [-0.0219, -0.0721, -0.0695,  ...,  0.0811,  0.1278, -0.0460],
        [-0.0915, -0.0807,  0.0212,  ..., -0.1490, -0.0332, -0.0137]],
       device='cuda:0')
torch.float32
tensor([ 6.5213e-02, -1.4691e-02, -6.5551e-02,  2.5778e-03,  2.4274e-02,
        -5.5316e-02, -1.3256e-02, -6.2497e-02,  4.8796e-03,  2.3916e-02,
         5.7960e-02,  2.6246e-02, -4.1447e-02, -5.2343e-02, -1.1208e-01,
        -3.1156e-02, -1.0041e-02,  3.2715e-02,  7.3524e-02,  4.4894e-02,
        -3.0859e-02,  5.6301e-02,

In [ ]:
def test_model_rounded(model, test_dataset, batch_size):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    test_loader = DataLoader(ImageDataset(test_dataset), batch_size=batch_size, shuffle=False, num_workers=1)

    model.eval()
    model.to(device)

    criterion = nn.CrossEntropyLoss().to(device)

    total_loss, total_acc = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

    avg_loss = total_loss / len(test_loader.dataset)
    avg_acc = total_acc / len(test_loader.dataset)
    print(f'Test - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return avg_loss, avg_acc

# Test the model
test_loss, test_accuracy = test_model_rounded(rounded_model, dataset['test'], BATCH_SIZE)
print(test_loss, test_accuracy)

In [17]:
import torch
import os
model.to("cpu")
# Assuming 'model' is your original ViT model and 'quantized_model' is the quantized version

# Save the original model
torch.save(trained_model.state_dict(), 'original_model.pth')

# Save the quantized model
torch.save(rounded_model.state_dict(), 'rounded_model.pth')

# Function to get file size in MB
def file_size_in_mb(file_path):
    size_bytes = os.path.getsize(file_path)
    return size_bytes / (1024 * 1024)  # Convert bytes to megabytes

# Print the file sizes
original_size = file_size_in_mb('original_model.pth')
rounded_model_size = file_size_in_mb('rounded_model.pth')
print(f"Original Model Size: {original_size:.2f} MB")
print(f"Rounded Model Size: {rounded_model_size:.2f} MB")

Original Model Size: 327.43 MB
Rounded Model Size: 163.76 MB


# Huffman encoding

In [ ]:
# Collect encoding statistics


device = torch.device("cpu")
rounded_model.to(device)


all_weights_flat = []


for name, param in rounded_model.named_parameters():
    layer_weights = param.data.cpu().numpy().flatten().tolist()
    all_weights_flat.extend(layer_weights)


frequencies = Counter(all_weights_flat)
codec = HuffmanCodec.from_frequencies(frequencies)

print(frequencies)

In [19]:
huffman_model = copy.deepcopy(rounded_model)
encoded_weights_dict = {}

for name, param in huffman_model.named_parameters():
    layer_weights = param.data.flatten().tolist()
    encoded_weights = codec.encode(layer_weights)
    encoded_weights_dict[name] = encoded_weights
    decoded_weights = codec.decode(encoded_weights)
    assert layer_weights == decoded_weights, "Mismatch in encoding/decoding!"

torch.save(encoded_weights_dict, 'encoded_model_weights.pth')

In [ ]:

huffman_size = file_size_in_mb('encoded_model_weights.pth')
print(f"Huffman Model Size: {huffman_size:.2f} MB")

In [ ]:
for k in iter(encoded_weights_dict):
  print(k)

# Test model

In [ ]:
def test_model_huffman(model, test_dataset, batch_size, codec, encoded_weights_file):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    test_loader = DataLoader(ImageDataset(test_dataset), batch_size=batch_size, shuffle=False, num_workers=1)

    model = model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)

    total_loss, total_acc = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

    avg_loss = total_loss / len(test_loader.dataset)
    avg_acc = total_acc / len(test_loader.dataset)
    print(f'Test - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return avg_loss, avg_acc, model.state_dict()

## Test entire decoded Huffman model

In [ ]:
encoded_weights_dict = torch.load('encoded_model_weights.pth')
model = ViT()
model.to(device)

for name, param in model.named_parameters():
    if name in encoded_weights_dict:
        decoded_weights = codec.decode(encoded_weights_dict[name])
        decoded_weights_tensor = torch.tensor(decoded_weights, device=device).view_as(param.data)
        param.data.copy_(decoded_weights_tensor)
    else:
        print(f"Weight for layer {name} not found in encoded weights.")

In [ ]:
model.eval()
test_loss, test_accuracy, state_dic = test_model_huffman(model, dataset['test'], BATCH_SIZE, codec, 'encoded_model_weights.pth')
print(test_loss, test_accuracy)

## Test dynamically decoded huffman model

In [ ]:
from transformers.models.vit.modeling_vit import ViTAttention, ViTLayer, ViTIntermediate, ViTOutput

def apply_huffman_decoding_to_layer(module: nn.Module, encoded_weights: dict, codec: HuffmanCodec, layer_path='') -> None:

    for attr_str in dir(module):
        target_attr = getattr(module, attr_str)
        if isinstance(module, ViTAttention):
            for component_name in ['query', 'key', 'value']:
                component = getattr(module.attention, component_name)
                for param in ['weight', 'bias']:
                    layer_name = f"{layer_path}attention.{component_name}.{param}"
                    if layer_name in encoded_weights:
                        encoded = encoded_weights[layer_name]
                        decoded_weights = codec.decode(encoded)
                        decoded_tensor = torch.tensor(decoded_weights, device=getattr(component, param).device).view_as(getattr(component, param))
                        getattr(component, param).data.copy_(decoded_tensor)
                        print(f"Updated {layer_name}")

            # Process the output dense layer of ViTAttention
            for param in ['weight', 'bias']:
                layer_name = f"{layer_path}output.dense.{param}"
                if layer_name in encoded_weights:
                    param_attr = getattr(module.output.dense, param)
                    encoded = encoded_weights[layer_name]
                    decoded_weights = codec.decode(encoded)
                    decoded_tensor = torch.tensor(decoded_weights, device=param_attr.device).view_as(param_attr)
                    param_attr.data.copy_(decoded_tensor)
                    print(f"Updated {layer_name}")

        elif isinstance(module, ViTIntermediate):
            # Process ViTIntermediate layers
            for param in ['weight', 'bias']:
                layer_name = f"{layer_path}dense.{param}"
                if layer_name in encoded_weights:
                    param_attr = getattr(module.dense, param)
                    encoded = encoded_weights[layer_name]
                    decoded_weights = codec.decode(encoded)
                    decoded_tensor = torch.tensor(decoded_weights, device=param_attr.device).view_as(param_attr)
                    param_attr.data.copy_(decoded_tensor)
                    print(f"Updated {layer_name}")

        elif isinstance(module, ViTOutput):
            # Process ViTOutput layers
            for param in ['weight', 'bias']:
                layer_name = f"{layer_path}dense.{param}"
                if layer_name in encoded_weights:
                    param_attr = getattr(module.dense, param)
                    encoded = encoded_weights[layer_name]
                    decoded_weights = codec.decode(encoded)
                    decoded_tensor = torch.tensor(decoded_weights, device=param_attr.device).view_as(param_attr)
                    param_attr.data.copy_(decoded_tensor)
                    print(f"Updated {layer_name}")

        elif 'layernorm' in layer_path:
            # Handle layers in vit.layernorm
            for param_name, param in module.named_parameters(recurse=False):
                full_param_name = f"{layer_path}{param_name}"
                if full_param_name in ['vit.layernorm.weight', 'vit.layernorm.bias']:
                    if full_param_name in encoded_weights:
                        encoded = encoded_weights[full_param_name]
                        decoded_weights = codec.decode(encoded)
                        decoded_tensor = torch.tensor(decoded_weights, device=param.device).view_as(param)
                        param.data.copy_(decoded_tensor)
                        print(f"Updated {full_param_name}")

        elif 'classifier' in layer_path:
            # Handle layers in classifier
            for param_name, param in module.named_parameters(recurse=False):
                full_param_name = f"{layer_path}{param_name}"
                if full_param_name in ['classifier.weight', 'classifier.bias']:
                    if full_param_name in encoded_weights:
                        encoded = encoded_weights[full_param_name]
                        decoded_weights = codec.decode(encoded)
                        decoded_tensor = torch.tensor(decoded_weights, device=param.device).view_as(param)
                        param.data.copy_(decoded_tensor)
                        print(f"Updated {full_param_name}")

        elif 'embeddings' in layer_path:
            # Handle specific layers in vit.embeddings
            for param_name, param in module.named_parameters(recurse=False):
                full_param_name = f"{layer_path}{param_name}"
                if full_param_name in ['vit.embeddings.cls_token', 'vit.embeddings.position_embeddings',
                                      'vit.embeddings.patch_embeddings.projection.weight',
                                      'vit.embeddings.patch_embeddings.projection.bias']:
                    if full_param_name in encoded_weights:
                        encoded = encoded_weights[full_param_name]
                        decoded_weights = codec.decode(encoded)
                        decoded_tensor = torch.tensor(decoded_weights, device=param.device).view_as(param)
                        param.data.copy_(decoded_tensor)
                        print(f"Updated {full_param_name}")

    for name, child in module.named_children():
        new_layer_path = f"{layer_path}{name}." if layer_path else f"{name}."
        apply_huffman_decoding_to_layer(child, encoded_weights, codec, new_layer_path)


In [ ]:
import torch
import torch.nn as nn
from transformers import ViTModel, ViTConfig
from transformers import ViTModel, ViTConfig

class ViT(nn.Module):
    def __init__(self, config=ViTConfig(), num_labels=20,
                 model_checkpoint='google/vit-base-patch16-224-in21k',
                 encoded_weights=None, codec=None):
        super(ViT, self).__init__()
        self.vit = ViTModel.from_pretrained(model_checkpoint, add_pooling_layer=False)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.encoded_weights = encoded_weights
        self.codec = codec

    def forward(self, x):
        if hasattr(self, 'encoded_weights') and hasattr(self, 'codec'):
            apply_huffman_decoding_to_layer(self, self.encoded_weights, self.codec)

        x = self.vit(x)['last_hidden_state']
        output = self.classifier(x[:, 0, :])
        return output


In [27]:
encoded_weights_dict = torch.load('encoded_model_weights.pth')
model_new = ViT(encoded_weights=encoded_weights_dict, codec=codec)
model_new.to(device)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ViT(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072,

In [ ]:
model_new.eval()
test_loss, test_accuracy, state_dic = test_model_huffman(model_new, dataset['test'], BATCH_SIZE, codec, 'encoded_model_weights.pth')
print(test_loss, test_accuracy)

In [ ]:
import torch
import torch.nn as nn
from transformers import ViTModel, ViTConfig
from transformers.models.vit.modeling_vit import ViTAttention, ViTLayer, ViTIntermediate, ViTOutput

class HuffmanDecodedLayer(nn.Module):
    """ A module wrapper to apply Huffman decoding to individual layers of a ViT model. """
    def __init__(self, original_module, encoded_weights, codec, layer_path=''):
        super().__init__()
        self.module = original_module
        self.encoded_weights = encoded_weights
        self.codec = codec
        self.layer_path = layer_path
        self.original_weights = {}  # To store original weights

    def apply_decoding(self, module, layer_path):

        """ Applies Huffman decoding to the weights of the module. """

        if isinstance(module, ViTAttention):
            for component_name in ['query', 'key', 'value']:
                component = getattr(module.attention, component_name)
                for param in ['weight', 'bias']:
                    layer_name = f"{layer_path}attention.{component_name}.{param}"
                    if layer_name in encoded_weights:

                        # TODO: Store original weight for other layers, add to other elifs
                        original_weight = getattr(component, param).data.clone()
                        self.original_weights[layer_name] = original_weight

                        encoded = encoded_weights[layer_name]
                        decoded_weights = codec.decode(encoded)
                        decoded_tensor = torch.tensor(decoded_weights, device=getattr(component, param).device).view_as(getattr(component, param))
                        getattr(component, param).data.copy_(decoded_tensor)
                        print(f"Updated {layer_name}")

            # Process the output dense layer of ViTAttention
            for param in ['weight', 'bias']:
                layer_name = f"{layer_path}output.dense.{param}"
                if layer_name in encoded_weights:
                    param_attr = getattr(module.output.dense, param)
                    encoded = encoded_weights[layer_name]
                    decoded_weights = codec.decode(encoded)
                    decoded_tensor = torch.tensor(decoded_weights, device=param_attr.device).view_as(param_attr)
                    param_attr.data.copy_(decoded_tensor)
                    print(f"Updated {layer_name}")

        elif isinstance(module, ViTIntermediate):
            # Process ViTIntermediate layers
            for param in ['weight', 'bias']:
                layer_name = f"{layer_path}dense.{param}"
                if layer_name in encoded_weights:
                    param_attr = getattr(module.dense, param)
                    encoded = encoded_weights[layer_name]
                    decoded_weights = codec.decode(encoded)
                    decoded_tensor = torch.tensor(decoded_weights, device=param_attr.device).view_as(param_attr)
                    param_attr.data.copy_(decoded_tensor)
                    print(f"Updated {layer_name}")

        elif isinstance(module, ViTOutput):
            # Process ViTOutput layers
            for param in ['weight', 'bias']:
                layer_name = f"{layer_path}dense.{param}"
                if layer_name in encoded_weights:
                    param_attr = getattr(module.dense, param)
                    encoded = encoded_weights[layer_name]
                    decoded_weights = codec.decode(encoded)
                    decoded_tensor = torch.tensor(decoded_weights, device=param_attr.device).view_as(param_attr)
                    param_attr.data.copy_(decoded_tensor)
                    print(f"Updated {layer_name}")

        elif 'layernorm' in layer_path:
            # Handle layers in vit.layernorm
            for param_name, param in module.named_parameters(recurse=False):
                full_param_name = f"{layer_path}{param_name}"
                if full_param_name in ['vit.layernorm.weight', 'vit.layernorm.bias']:
                    if full_param_name in encoded_weights:
                        encoded = encoded_weights[full_param_name]
                        decoded_weights = codec.decode(encoded)
                        decoded_tensor = torch.tensor(decoded_weights, device=param.device).view_as(param)
                        param.data.copy_(decoded_tensor)
                        print(f"Updated {full_param_name}")

        elif 'classifier' in layer_path:
            # Handle layers in classifier
            for param_name, param in module.named_parameters(recurse=False):
                full_param_name = f"{layer_path}{param_name}"
                if full_param_name in ['classifier.weight', 'classifier.bias']:
                    if full_param_name in encoded_weights:
                        encoded = encoded_weights[full_param_name]
                        decoded_weights = codec.decode(encoded)
                        decoded_tensor = torch.tensor(decoded_weights, device=param.device).view_as(param)
                        param.data.copy_(decoded_tensor)
                        print(f"Updated {full_param_name}")

        elif 'embeddings' in layer_path:
            # Handle specific layers in vit.embeddings
            for param_name, param in module.named_parameters(recurse=False):
                full_param_name = f"{layer_path}{param_name}"
                if full_param_name in ['vit.embeddings.cls_token', 'vit.embeddings.position_embeddings',
                                      'vit.embeddings.patch_embeddings.projection.weight',
                                      'vit.embeddings.patch_embeddings.projection.bias']:
                    if full_param_name in encoded_weights:
                        encoded = encoded_weights[full_param_name]
                        decoded_weights = codec.decode(encoded)
                        decoded_tensor = torch.tensor(decoded_weights, device=param.device).view_as(param)
                        param.data.copy_(decoded_tensor)
                        print(f"Updated {full_param_name}")


    # TODO: implement for other layers, not only ViTAttention
    def restore_original_weights(self, module, layer_path):
        if isinstance(module, ViTAttention):
            for component_name in ['query', 'key', 'value']:
                component = getattr(module.attention, component_name)
                for param in ['weight', 'bias']:
                    layer_name = f"{layer_path}attention.{component_name}.{param}"
                    if layer_name in self.original_weights:
                        original_weight = self.original_weights[layer_name]
                        getattr(component, param).data.copy_(original_weight)


    def forward(self, *args, **kwargs):
        self.apply_decoding(self.module, self.layer_path)
        output = self.module(*args, **kwargs)
        self.restore_original_weights(self.module, self.layer_path)
        return output


class CustomViT(nn.Module):
    def __init__(self, config=ViTConfig(), num_labels=20,
                 model_checkpoint='google/vit-base-patch16-224-in21k',
                 encoded_weights=None, codec=None):
        super(CustomViT, self).__init__()
        self.vit = ViTModel.from_pretrained(model_checkpoint, add_pooling_layer=False)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.encoded_weights = encoded_weights
        self.codec = codec

    def forward(self, x):
        x = self.apply_huffman_decoding(self.vit, x, self.encoded_weights, self.codec)
        output = self.classifier(x[:, 0, :])
        return output

    def apply_huffman_decoding(self, module, x, encoded_weights, codec, layer_path=''):
        for name, child in module.named_children():
            new_layer_path = f"{layer_path}{name}." if layer_path else f"{name}."
            if isinstance(child, (ViTAttention, ViTIntermediate, ViTOutput)) or 'layernorm' in layer_path or 'classifier' in layer_path or 'embeddings' in layer_path:
                wrapped_layer = HuffmanDecodedLayer(child, encoded_weights, codec, new_layer_path)
                x = wrapped_layer(x)
            else:
                x = self.apply_huffman_decoding(child, x, encoded_weights, codec, new_layer_path)
        return x


In [ ]:
encoded_weights_dict = torch.load('encoded_model_weights.pth')

config = ViTConfig()
num_labels = 20
model_checkpoint = 'google/vit-base-patch16-224-in21k'
custom_vit = CustomViT(config=config, num_labels=num_labels, model_checkpoint=model_checkpoint, encoded_weights=encoded_weights, codec=codec)

custom_vit = CustomViT(encoded_weights=encoded_weights_dict, codec=codec)
custom_vit.to(device)

custom_vit.eval()
test_loss, test_accuracy, state_dic = test_model_huffman(custom_vit, dataset['test'], BATCH_SIZE, codec, 'encoded_model_weights.pth')
print(test_loss, test_accuracy)

In [ ]:
import torch
import struct
import torch
import struct
import io

# The original float value is 123.456 -> its 4-byte representation is a byte string b'y\xe9\xf6B'.
def serialize_weights_pytorch(model):
    serialized_weights = []
    for param_tensor in model.state_dict():
        tensor = model.state_dict()[param_tensor].cpu().numpy()
        flattened_weights = tensor.flatten()
        for weight in flattened_weights:
            # Convert each weight to a 4-byte representation
            serialized_weights.append(struct.pack('f', weight))
    return b''.join(serialized_weights)


In [ ]:
# Updated LZW compression function to output a byte stream
def lzw_compress(uncompressed):
    dict_size = 256
    dictionary = {bytes([i]): i for i in range(dict_size)}

    w = b""
    result = io.BytesIO()
    for c in uncompressed:
        wc = w + bytes([c])
        if wc in dictionary:
            w = wc
        else:
            result.write(dictionary[w].to_bytes(2, byteorder='little'))
            dictionary[wc] = dict_size
            dict_size += 1
            w = bytes([c])

    if w:
        result.write(dictionary[w].to_bytes(2, byteorder='little'))
    return result.getvalue()

In [ ]:
# Assume 'model' is your pre-trained model
copy=trained_model
serialized = serialize_weights_pytorch(copy)

In [ ]:
print(serialized[0:100])

In [ ]:
compressed = lzw_compress(serialized)

In [ ]:
print(compressed[0:100])

In [ ]:
torch.save(compressed, 'compressed_lzw.pth')

In [ ]:
lzw_size = file_size_in_mb('compressed_lzw.pth')
print(f"LZW Model Size: {lzw_size:.2f} MB")